# Phase 1: Exploratory data analysis and data pre-processing

### Taryn Chung (301239706), ZeYu Zhu (301341176), Kearro Chow (301279961)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
color = sns.color_palette()

In [3]:
train_df = pd.read_json("train.json")
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,145 Borinquen Place,medium
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800,230 East 44th,low
9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495,405 East 56th Street,medium
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,medium
15,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,2c3b41f588fbb5234d8a1e885a436cfa,[https://photos.renthop.com/2/7225292_901f1984...,2795,340 East 34th Street,low


## Visualization of target variable (interest level)

In [ ]:
# int_level = train_df['interest_level'].value_counts()

# plt.figure(figsize=(6,6))
# sns.barplot(int_level.index, int_level.values, alpha=1, color=color[0])
# plt.ylabel('Number of Occurrences', fontsize=14, fontweight='bold')
# plt.xlabel('Interest level', fontsize=14, fontweight='bold')
# plt.xticks(fontsize=10, fontweight='bold')
# plt.yticks(fontsize=10, fontweight='bold')
# plt.show()

In [ ]:
# plt.pie(int_level.values,(0,0,0),int_level.index, autopct='%1.0f%%', 
#        shadow=False, startangle=0,   
#        pctdistance=1.2,labeldistance=1.4)
# plt.axis('equal')
# plt.show()

## Hour-wise listing trend

In [ ]:
# train_df["created"] = pd.to_datetime(train_df["created"])
# train_df["hour_created"] = train_df["created"].dt.hour
# hour = train_df['hour_created'].value_counts()
# plt.figure(figsize=(12,6))
# sns.barplot(hour.index, hour.values, alpha=0.8, color=color[1])
# plt.xlabel('hours', fontsize=12, fontweight='bold')
# plt.ylabel('# created', fontsize=12, fontweight='bold')
# plt.xticks(rotation='vertical', fontweight='bold')
# plt.yticks(fontweight='bold')
# plt.show()

In [ ]:
# top5 = hour[:5]
# print("Busiest 5 Hours:\n",top5)

## Visualizing features

In [ ]:
# plt.figure(figsize=(8,6))
# sns.distplot(train_df.price.values, bins=50, kde=False)
# plt.xlabel('price', fontsize=12, fontweight='bold')
# plt.xticks(fontweight='bold')
# plt.yticks(fontweight='bold')
# plt.show()

In [ ]:
# plt.figure(figsize=(8,6))
# sns.distplot(train_df.bathrooms.values, kde=False)
# plt.xlabel('bathroom', fontsize=12, fontweight='bold')
# plt.xticks(fontweight='bold')
# plt.yticks(fontweight='bold')
# plt.show()

In [ ]:
# plt.figure(figsize=(8,6))
# sns.distplot(train_df.bedrooms.values, kde=False)
# plt.xlabel('bedroom', fontsize=12, fontweight='bold')
# plt.xticks(fontweight='bold')
# plt.yticks(fontweight='bold')
# plt.show()

In [ ]:
# plt.figure(figsize=(8,6))
# sns.distplot(train_df.listing_id.values, kde=False)
# plt.xlabel('listing_id', fontsize=12, fontweight='bold')
# plt.xticks(fontweight='bold')
# plt.yticks(fontweight='bold')
# plt.show()

In [ ]:
# plt.figure(figsize=(8,6))
# sns.distplot(train_df.latitude.values,kde=False)
# plt.xlabel('latitude', fontsize=12, fontweight='bold')
# plt.xticks(fontweight='bold')
# plt.yticks(fontweight='bold')
# plt.show()

In [ ]:
# plt.figure(figsize=(8,6))
# sns.distplot(train_df.longitude.values, kde=False)
# plt.xlabel('longitude', fontsize=12, fontweight='bold')
# plt.xticks(fontweight='bold')
# plt.yticks(fontweight='bold')
# plt.show()


## Handling missing values

In [13]:
# check if there are any null/nan values
train_df.isnull().sum()

bathrooms          0
bedrooms           0
building_id        0
created            0
description        0
display_address    0
features           0
latitude           0
listing_id         0
longitude          0
manager_id         0
photos             0
price              0
street_address     0
interest_level     0
dtype: int64

### Bedrooms

In [ ]:
# aggregate bedroom values by count
train_df.groupby('bedrooms').count();

In [15]:
# sum count of all bedroom values
train_df.groupby('bedrooms').count().sum()

bathrooms          49352
building_id        49352
created            49352
description        49352
display_address    49352
features           49352
latitude           49352
listing_id         49352
longitude          49352
manager_id         49352
photos             49352
price              49352
street_address     49352
interest_level     49352
dtype: int64

We found 9475 values with a bedroom count of 0. This is simply too high of a number to assume it is an error. Upon checking some photos, we realized that these are probably studio apartments so no bedrooms makes sense. When we aggregated the count of every value of bedrooms, we found there were no missing values.

### Bathrooms

In [ ]:
# aggregate bathroom values by count
train_df.groupby('bathrooms').count();

In [17]:
# sum the count of all bathroom values
train_df.groupby('bathrooms').count().sum()

bedrooms           49352
building_id        49352
created            49352
description        49352
display_address    49352
features           49352
latitude           49352
listing_id         49352
longitude          49352
manager_id         49352
photos             49352
price              49352
street_address     49352
interest_level     49352
dtype: int64

Again, we found no missing values for bathrooms by the same technique as bedrooms. We found 313 places with with 0 bathrooms but this can be normal for places with a shared bathroom on a floor. 

### Building ID

In [18]:
whichrow = 0
count = 0
for row in train_df['building_id']:
    if row=='0' or any(c.isalpha() for c in row) == False:
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)
#train_df.head(30)

number of missing values:  8286


It's unlikely that building ID has any effect on the interest level of a listing since it is a random number that has no meaningful numerical value other than to identify the listing. We left these values since it is unlikely we will be training with this feature.

### Description

In [19]:
whichrow = 0
count = 0
for row in train_df['description']:
    if  any(c.isalpha() for c in row) == False:
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)
#train_df.head(50)

number of missing values:  3336


We left these values because having no description can also affect the interest level of a buyer.

### Features

In [20]:
whichrow = 0
count = 0
for row in train_df['features']:
    if row==[]:
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)
#train_df.head(50)

number of missing values:  3218


We left these values because having no features listed can also affect the interest level of a buyer.

### Created

In [21]:
import datetime

whichrow = 0
count = 0
for row in train_df['created']:
    if isinstance(row, datetime.datetime) == False:
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  49352


### Display Address

In [22]:
whichrow = 0
count = 0
for row in train_df['display_address']:
    if any(c.isalpha() for c in row) == False:
        train_df.loc[train_df.index[whichrow], 'display_address'] = np.nan
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  141


Since there are relatively few missing values and an address is important to have, we removed these values by changing them to NAN and dropping every row with a NAN.

### Street Address

In [23]:
whichrow = 0
count = 0
for row in train_df['street_address']:
    if any(c.isalpha() for c in row) == False :
        train_df.loc[train_df.index[whichrow], 'street_address'] = np.nan
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  13


Since there are relatively few missing values and an address is important to have, we removed these values by changing them to NAN and dropping every row with a NAN.

### Latitude

In [24]:
whichrow = 0
count = 0
for row in train_df['latitude']:
    if row==0:
        train_df.loc[train_df.index[whichrow], 'latitude'] = np.nan
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  12


Since there are relatively few missing values and an coordinates are important to have, we removed these values by changing them to NAN and dropping every row with a NAN.

### Listing ID

In [25]:
whichrow = 0
count = 0
for row in train_df['listing_id']:
    if row==0 or row==None:
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  0


### Longitude

In [26]:
whichrow = 0
count = 0
for row in train_df['longitude']:
    if row==0:
        train_df.loc[train_df.index[whichrow], 'longitude'] = np.nan
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  12


Since there are relatively few missing values and an coordinates are important to have, we removed these values by changing them to NAN and dropping every row with a NAN.

### Manager ID

In [27]:
whichrow = 0
count = 0
for row in train_df['manager_id']:
    if  row==0 or row =='0' or any(c.isalpha() for c in row) == False:
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  0


### Photos

In [28]:
whichrow = 0
count = 0
for row in train_df['photos']:
    if  any(isinstance(c,str) for c in row) == False:
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  3615


Since we won't be using this feature in later milestones, we left these missing values.

### Price

In [29]:
whichrow = 0
count = 0
for row in train_df['price']:
    if row==0:
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  0


In [ ]:
#double check missing values
train_df.groupby('price').count().sum();

### Interest Level

In [31]:
whichrow = 0
count = 0
list= ["low", "medium", "high"]
for row in train_df['interest_level']:
    if row not in list :
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  0


In [ ]:
#remove selected rows with few missing values
train_df = train_df.dropna()

## Handling outliers

### Prices

In [33]:
q25, q75 = np.percentile(train_df['price'].values, 25), np.percentile(train_df['price'].values, 75)
iqr = q75-q25
cutoff = iqr * 3
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['price'].values if x > upper or x < lower]
print ("number of extreme outliers:", len(outliers))
#print(lower,upper)

number of extreme outliers: 1220


In [34]:
q25, q75 = np.percentile(train_df['price'].values, 25), np.percentile(train_df['price'].values, 75)
iqr = q75-q25
cutoff = iqr * 1.5
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['price'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
#print(lower,upper)

number of mild outliers: 2783


#### Before Handling

In [ ]:
# sns.boxplot(y=train_df['price'])

#### After Handling

In [36]:
from scipy import nanmean
import math

whichrow = 0
for row in train_df['price']:
    if row>upper or row<lower:
        train_df.loc[train_df.index[whichrow], 'price'] = np.nan
        whichrow+=1
    else:
        whichrow += 1
        
mean_val =round(nanmean(train_df['price']),2)

whichrow = 0
for row in train_df['price']:
    if math.isnan(row):
        train_df.loc[train_df.index[whichrow],'price'] = mean_val
        whichrow+=1
    else:
        whichrow+=1

# sns.boxplot(y=train_df['price'], whis = [0,100])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: scipy.nanmean is deprecated and will be removed in SciPy 2.0.0, use numpy.nanmean instead
  if sys.path[0] == '':


We found the Tukey inner and outer fences to find the mild and extreme outliers. Extreme fences included a cutoff that left listings with a negative price range. We decided to only remove the mild outliers instead.  We replaced them with the mean value 3272.00 of so we have more data points to train with. The upper fence cutoff was 6,500 and the lower cutoff was 100. This dealt with outliers such as listings prices at 4,490,000, 43, and negative numbers, which are unrealistic. The new minimum price is 401 and the maximum is now 6,500.

### Latitude

#### Before Handling

In [37]:
q25, q75 = np.percentile(train_df['latitude'].values, 25), np.percentile(train_df['latitude'].values, 75)
iqr = q75-q25
cutoff = iqr * 1.5
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['latitude'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
#print(lower, upper)

number of mild outliers: 1910


In [38]:
q25, q75 = np.percentile(train_df['latitude'].values, 25), np.percentile(train_df['latitude'].values, 75)
iqr = q75-q25
cutoff = iqr * 3
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['latitude'].values if x > upper or x < lower]
print ("number of extreme outliers:", len(outliers))
#print(lower, upper)

number of extreme outliers: 135


In [ ]:
# sns.boxplot(y=train_df['latitude'])

#### After Handling

In [40]:
whichrow = 0
for row in train_df['latitude']:
    if row>upper or row<lower:
        train_df.loc[train_df.index[whichrow], 'latitude'] = np.nan
        whichrow+=1
    else:
        whichrow += 1
        
mean_val = nanmean(train_df['latitude'])
#print(mean_val)
whichrow = 0
for row in train_df['latitude']:
    if math.isnan(row):
        train_df.loc[train_df.index[whichrow],'latitude'] = mean_val
        whichrow+=1
    else:
        whichrow+=1

# sns.boxplot(y=train_df['latitude'], whis = [0,100])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: scipy.nanmean is deprecated and will be removed in SciPy 2.0.0, use numpy.nanmean instead
  if __name__ == '__main__':


We found the Tukey inner and outer fences to find the mild and extreme outliers. The inner fences still included listings that were within the NYC bounds, accoding to Google Maps. So we decided to only deal with the extreme outliers that were a little further out than NYC vicinity. We replaced with the mean value  of 40.7513098874893 so we have more data points to train with. The fence cutoffs we used were 40.5903 and 40.912299999999995. This dealt with outliers such as listings located in LA because we wanted to keep the listings within NYC. The new maximum and minimum longitude and latitude are now 40.9121 and 40.5904, respectively.

### Longitude

In [41]:
q25, q75 = np.percentile(train_df['longitude'].values, 25), np.percentile(train_df['longitude'].values, 75)
iqr = q75-q25
cutoff = iqr * 3
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['longitude'].values if x > upper or x < lower]
print ("number of extreme outliers:", len(outliers))
#print(lower, upper)

number of extreme outliers: 399


In [42]:
q25, q75 = np.percentile(train_df['longitude'].values, 25), np.percentile(train_df['longitude'].values, 75)
iqr = q75-q25
cutoff = iqr * 1.5
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['longitude'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
#print(lower, upper)

number of mild outliers: 1087


#### Before Handling

In [ ]:
# sns.boxplot(y=train_df['longitude'])

#### After Handling

In [44]:
whichrow = 0
for row in train_df['longitude']:
    if row>upper or row<lower:
        train_df.loc[train_df.index[whichrow], 'longitude'] = np.nan
        whichrow+=1
    else:
        whichrow += 1
        
mean_val = nanmean(train_df['longitude'])
print(mean_val)

whichrow = 0
for row in train_df['longitude']:
    if math.isnan(row):
        train_df.loc[train_df.index[whichrow],'longitude'] = mean_val
        whichrow+=1
    else:
        whichrow+=1

# sns.boxplot(y=train_df['longitude'] , whis = [0,100])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: scipy.nanmean is deprecated and will be removed in SciPy 2.0.0, use numpy.nanmean instead
  if __name__ == '__main__':


-73.97499519850344


We found the Tukey inner and outer fences to find the mild and extreme outliers. The outer fences still included listings that were within the NYC borough bounds, accoding to Google Maps. The very bottom tip of New York the island is around -74.01 and -73.88 is in the Bronx. So we decided to only deal with the mild outliers that encompassed this. We replaced with the mean value  of 40.7513098874893 so we have more data points to train with. The fence cutoffs we used were -74.04704999999998 and -73.89945000000003. This dealt with outliers such as listings located in LA because we wanted to keep the listings within NYC. The new minimum and maximum longitude and latitude are now -74.0454 and -73.8995, respectively.

### Bedrooms

In [45]:
q25, q75 = np.percentile(train_df['bedrooms'].values, 25), np.percentile(train_df['bedrooms'].values, 75)
iqr = q75-q25
cutoff = iqr * 1.5
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['bedrooms'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
print(lower, upper)

number of mild outliers: 2220
-0.5 3.5


In [46]:
q25, q75 = np.percentile(train_df['bedrooms'].values, 25), np.percentile(train_df['bedrooms'].values, 75)
iqr = q75-q25
cutoff = iqr * 3
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['bedrooms'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
print(lower, upper)

number of mild outliers: 49
-2.0 5.0


#### Before Handling

In [ ]:
# sns.boxplot(y=train_df['bedrooms'])

We found the Tukey inner and outer fences to find the mild and extreme outliers. There were 'outliers' but looking at the values, they seem to make sense, given the prices of our listings. It would make sense for a penthouse or large properties to have many bedrooms so a value like 8 isn't an error. We decided to leave this many values unchanged.

### Bathroom

In [48]:
q25, q75 = np.percentile(train_df['bathrooms'].values, 25), np.percentile(train_df['bathrooms'].values, 75)
iqr = q75-q25
cutoff = iqr * 1.5
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['bathrooms'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
print(lower,upper)

number of mild outliers: 9911
1.0 1.0


In [49]:
q25, q75 = np.percentile(train_df['bathrooms'].values, 25), np.percentile(train_df['bathrooms'].values, 75)
iqr = q75-q25
cutoff = iqr * 3
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['bathrooms'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
print(lower,upper)

number of mild outliers: 9911
1.0 1.0


In [ ]:
# sns.boxplot(y=train_df['bathrooms'])

We found the Tukey inner and outer fences to find the mild and extreme outliers. A quarter of the data seem to be 'outliers' because the distriution is so largge, the upper and lower cutoffs were both 1. It doesn't make sense to remove a quarter of our data. Looking at the values, they seem to make sense, given the prices of our listings. It would make sense for a penthouse or large properties to have many bathrooms. We decided to leave this many values unchanged because this is unlikely due to noise.

<!-- ### Bathroom and Bedrooms

Outlier detection for bathrooms and bedrooms isn't meaningful because all values fall within a tidy distribution. All bathroom values lie between 0-4 and all bedroom values lie between 0-3, which makes sense for studio apartments and shared bathrooms. -->

### Created

Since no missing values were found, it doesn't seem meaningful to detect outliers since the only outlier values would be missing values or dates that don't exist (not one of 365 days in a year).

### Building ID, Manager ID, Listing ID

Again, not meaningful for outliers because there is no real meaning to these values, other than to identify the listing.

### Photos, Description, Features, Addresses

No discernable way to identify outliers until we are able to extract these features. Outliers would probably be photos of objects other than the listing, and text that doesn't relate, or an address outside of New York.

## Text feature extraction

Here we expore the text in two meaningful features of the listings: features and description. We use the TFIDF to tokenize the words found in each feature. Common stop words were excluded and we defined each word 2+ characters because there were some words found such as 'xx' and other random strings. Since the features are actually lists in this column, we concatenated them to string before tokenize. After tokenization, we vectorize each feature into new columns. 

In [ ]:
# #description
# vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', token_pattern=r'^[a-zA-Z][a-zA-Z]+')
# vectorizer.fit(train_df['description'].values);
# train_df['desc_vect'] = train_df['description'].apply(lambda x: vectorizer.transform([x]))
# # vectorizer.vocabulary_  #shows all tokens

In [ ]:
# convert features list to string
train_df['features'] = train_df['features'].apply(lambda x: ' '.join(x))

In [ ]:
# # #features
# vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', token_pattern=r'^[a-zA-Z][a-zA-Z]+')
# vectorizer.fit(train_df['features']);
# train_df['feat_vect'] = train_df['features'].apply(lambda x: vectorizer.transform([x]))
# # train_df['feat_vect'] = train_df['feat_vect'].apply(lambda x: x.toarray())
# # train_df['feat_vect'] = train_df['features'].apply(lambda x: vectorizer.transform(x).toarray())
# # # vectorizer.vocabulary_  #shows all tokens

In [ ]:
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', token_pattern=r'^[a-zA-Z][a-zA-Z]+')
# vectorizer = CountVectorizer(analyzer='word', stop_words='english', token_pattern=r'^[a-zA-Z][a-zA-Z]+')
w = vectorizer.fit_transform(train_df['features']);
# z = w.toarray()
# s = np.array(z)
# train_df['feat_vect'] = train_df.apply(lambda x: tuple(x), axis=1).apply(w)
# train_df.insert('feat_vect', w.toarray())
tempd = pd.DataFrame(w.toarray(), columns=vectorizer.get_feature_names())
train_df.reset_index(inplace=True, drop=True)
train_df = pd.concat([train_df, tempd], axis=1)

## SVM

In [55]:
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level,absolute,actual,air,assigned,available,balcony,bike,brand,building,cable,cats,childrens,close,common,courtyard,deck,deco,decorative,dining,dishwasher,dogs,doorman,duplex,dw,eat,...,hardwood,high,laundry,live,loft,low,multi,new,newly,nyu,offering,outdoor,parking,patio,pets,pool,postwar,pre,prewar,private,queen,recreational,reduced,renovated,residents,roof,short,stainless,storage,swimming,terrace,townhouse,triplex,view,walk,washer,wifi,wonderful,wood,yard
0,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,Dining Room Pre-War Laundry in Building Dishwa...,40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400.0,145 Borinquen Place,medium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,Doorman Elevator Laundry in Building Dishwashe...,40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800.0,230 East 44th,low,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,Doorman Elevator Laundry in Building Laundry i...,40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495.0,405 East 56th Street,medium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000.0,792 Metropolitan Avenue,medium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,Doorman Elevator Fitness Center Laundry in Bui...,40.7439,7225292,-73.9743,2c3b41f588fbb5234d8a1e885a436cfa,[https://photos.renthop.com/2/7225292_901f1984...,2795.0,340 East 34th Street,low,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
train_df['interest_level'] = train_df['interest_level'].astype('category')
train_df['target'] = train_df['interest_level'].cat.codes
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level,absolute,actual,air,assigned,available,balcony,bike,brand,building,cable,cats,childrens,close,common,courtyard,deck,deco,decorative,dining,dishwasher,dogs,doorman,duplex,dw,eat,...,high,laundry,live,loft,low,multi,new,newly,nyu,offering,outdoor,parking,patio,pets,pool,postwar,pre,prewar,private,queen,recreational,reduced,renovated,residents,roof,short,stainless,storage,swimming,terrace,townhouse,triplex,view,walk,washer,wifi,wonderful,wood,yard,target
0,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,Dining Room Pre-War Laundry in Building Dishwa...,40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400.0,145 Borinquen Place,medium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,Doorman Elevator Laundry in Building Dishwashe...,40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800.0,230 East 44th,low,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,Doorman Elevator Laundry in Building Laundry i...,40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495.0,405 East 56th Street,medium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000.0,792 Metropolitan Avenue,medium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,Doorman Elevator Fitness Center Laundry in Bui...,40.7439,7225292,-73.9743,2c3b41f588fbb5234d8a1e885a436cfa,[https://photos.renthop.com/2/7225292_901f1984...,2795.0,340 East 34th Street,low,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.utils import shuffle

scaling = MinMaxScaler()
scaling2 = MaxAbsScaler()
scaling3 = Normalizer()
pca = PCA(n_components=1)

In [58]:
## Determine which features are more important for weighting 

x = train_df[['price']]
y = train_df['target']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.5, stratify=y) 
x_train = scaling3.fit_transform(x_train)
x_test = scaling3.fit_transform(x_test)
abc = SVC(kernel='linear', decision_function_shape = 'ovo', probability = True, cache_size=1000)
abc.fit(x_train, y_train)
y_pred = abc.predict(x_test)
y_pred2 = abc.predict_proba(x_test)
print('price \n')
print('log_loss: ',log_loss(y_test,y_pred2))

x = train_df[['bedrooms', 'bathrooms']]
y = train_df['target']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.5, stratify=y) 
x_train = scaling2.fit_transform(x_train)
x_test =  scaling2.fit_transform(x_test)
# train.transform(x_train)
# test.transform(x_test)
# x_train = scaling.fit_transform(x_train)
# x_test = scaling.fit_transform(x_test)
abc = SVC(kernel='linear', probability = True, cache_size=1000)
abc.fit(x_train, y_train)
y_pred = abc.predict(x_test)
y_pred2 = abc.predict_proba(x_test)
print('bedrooms, bathrooms \n')
print('log_loss: ',log_loss(y_test,y_pred2))

x = train_df[['latitude', 'longitude']]
y = train_df['target']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.5, stratify=y) 
x_train = scaling2.fit_transform(x_train)
x_test =  scaling2.fit_transform(x_test)
# x_train = scaling.fit_transform(x_train)
# x_test = scaling.fit_transform(x_test)
abc = SVC(kernel='linear', probability = True, cache_size=1000)
abc.fit(x_train, y_train)
y_pred = abc.predict(x_test)
y_pred2 = abc.predict_proba(x_test)
print('latitude, longitude \n')
print('log_loss: ',log_loss(y_test,y_pred2))

feat_d = ['interest_level','building_id','created', 'description','display_address','features', 'listing_id','manager_id','photos', 'street_address', 'target','price','bedrooms','bathrooms','longitude','latitude' ]
x = train_df.drop(feat_d, axis=1)
y = train_df['target']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.5, stratify=y) 
x_train = scaling2.fit_transform(x_train)
x_test =  scaling2.fit_transform(x_test)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)
abc = SVC(kernel='linear', decision_function_shape = 'ovo', probability = True, cache_size=1000)
abc.fit(x_train, y_train)
y_pred = abc.predict(x_test)
y_pred2 = abc.predict_proba(x_test)
print('PCA_Tfid \n')
print('log_loss: ',log_loss(y_test,y_pred2))


price 

log_loss:  0.7883159655023315
bedrooms, bathrooms 

log_loss:  0.7883299169907907
latitude, longitude 

log_loss:  0.7887565146847447
PCA_Tfid 

log_loss:  1.5678428502994486


# PCA_Tfid has terrible log_loss and therefore removed.

In [ ]:

# ## reduce majority sample

# train_df.reset_index(drop=True, inplace=True)
# # print(train_df)
# minority1 = train_df[train_df['interest_level'] == 'high']
# minority2 = train_df[train_df['interest_level'] == 'medium']
# majority = train_df[train_df['interest_level'] == 'low']

# print(minority1.shape)
# print(minority2.shape)
# print(majority.shape)

# # remove_n1 = 11490*2
# remove_n1 = 30362
# # remove_n2 = 3690
# remove_n2 = 7381
# drop_indices1 = np.random.choice(majority.index, remove_n1, replace=False)
# drop_indices2 = np.random.choice(minority2.index, remove_n2, replace=False)
# train_df = train_df.drop(drop_indices1)
# train_df = train_df.drop(drop_indices2)
# print(train_df.shape)

In [ ]:
# train_df.reset_index(drop=True, inplace=True)
# # print(train_df)
# minority1 = train_df[train_df['interest_level'] == 'high']
# minority2 = train_df[train_df['interest_level'] == 'medium']
# majority = train_df[train_df['interest_level'] == 'low']

# print(minority1.shape)
# print(minority2.shape)
# print(majority.shape)

# # remove_n1 = 11490*2
# remove_n1 = 30362
# # remove_n2 = 3690
# remove_n2 = 7381
# drop_indices1 = np.random.choice(majority.index, remove_n1, replace=False)
# drop_indices2 = np.random.choice(minority2.index, remove_n2, replace=False)
# train_df = train_df.drop(drop_indices1)
# train_df = train_df.drop(drop_indices2)

# print(train_df.shape)
# feat = ['price','bedrooms','bathrooms','longitude','latitude', 'feat_vect']
# x = train_df[feat]
# feat1 = ['interest_level','building_id','created', 'description','display_address','features', 'listing_id','manager_id','photos', 'street_address', 'target' ]


feat = ['price','bedrooms','bathrooms','longitude','latitude']
x= train_df[feat]
# x = train_df.drop(feat1,axis=1)
y = train_df['target']

# x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, stratify=y) 
# x_train, x_test = x.loc[ctrain_index[0]:ctrain_index[-1]], x.loc[ctest_index[0]:ctest_index[-1]]
# y_train, y_test = y.loc[ctrain_index[0]:ctrain_index[-1]], y.loc[ctest_index[0]:ctest_index[-1]]
# x_train.shape
# print(y_train)
# print(x_train.shape)

In [71]:
# feat2 = ['price','bedrooms','bathrooms','longitude','latitude']
# x_train[feat2]=scaling.fit_transform(x_train[feat2])
# x_test[feat2]=scaling.fit_transform(x_test[feat2])

x = scaling2.fit_transform(x)
# x_train = scaling2.fit_transform(x_train)
# x_test = scaling2.fit_transform(x_test)

# pca = PCA(n_components=6)
# x = pca.fit_transform(x)
# x_train = pca.fit_transform(x_train)
# x_test = pca.transform(x_test)

# quantile_transformer = QuantileTransformer(random_state=0)
# x_train = quantile_transformer.fit_transform(x_train)
# x_test = quantile_transformer.fit_transform(x_test)

# t1 = RobustScaler().fit(x_train)
# t2 = RobustScaler().fit(x_test)
# x_train = t1.transform(x_train)
# x_test = t2.transform(x_test)

# print(x_train.size)
# print(x_train)
svc_class = SVC(kernel='poly', probability = True, cache_size=1000)

# probability = True
# svc_class.fit(x_train, y_train)
svc_class.fit(x, y)

SVC(C=1.0, break_ties=False, cache_size=1000, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
# y_pred = svc_class.predict(x_test)
# y_pred2 = svc_class.predict_proba(x_test)
# y_pred

In [ ]:
# print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100)
# print('log_loss: ',log_loss(y_test,y_pred2))
# print(confusion_matrix(y_test,y_pred))
# print(classification_report(y_test,y_pred))

In [75]:
## Cross Validation

scaling = MinMaxScaler()

# train_df = train_df.sample(frac=1).reset_index(drop=True)
train_df = shuffle(train_df)
train_df.reset_index(inplace=True, drop=True)
# x = train_df.drop(feat1, axis=1)
x = train_df[feat]
# x = train_df[feat]
y = train_df['target']
# feat = ['price','bedrooms','bathrooms','longitude','latitude']
# x = train_df.loc[:, feat].values
# y = train_df.loc[:, ['target']].values

kf = KFold(n_splits=5)
i=1
# clf = SVC(kernel='linear', probability = True)
bestloss = 100
tsum = 0
for train_index, test_index in kf.split(x):
    # print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x.loc[train_index[0]:train_index[-1]], x.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y.loc[train_index[0]:train_index[-1]], y.loc[test_index[0]:test_index[-1]]
#     x_train = quantile_transformer.fit_transform(x_train)
#     x_test = quantile_transformer.fit_transform(x_test)
    x_train = scaling2.fit_transform(x_train)
    x_test = scaling2.fit_transform(x_test)
    # pca = PCA(n_components=6)
    # x_train = pca.fit_transform(x_train)
    # x_test = pca.transform(x_test)
    clf = SVC(kernel='poly', probability = True, cache_size=1000)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    # print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    # if log_loss(y_test,y_pred) <= bestloss:
    #   bestloss = log_loss(y_test,y_pred)
    #   ctrain_index = train_index
    #   ctest_index = test_index
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
# print('chosen train: ', ctrain_index, 'chosen test: ', ctest_index,'\n' )   
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.765016612088071 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       757
           1       0.70      1.00      0.82      6891
           2       0.00      0.00      0.00      2192

    accuracy                           0.70      9840
   macro avg       0.23      0.33      0.27      9840
weighted avg       0.49      0.70      0.58      9840

SVM Accuracy Score ->  70.03048780487805 for  1 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


log_loss for  2  :  0.7669702644637562 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       758
           1       0.70      1.00      0.82      6842
           2       0.00      0.00      0.00      2240

    accuracy                           0.70      9840
   macro avg       0.23      0.33      0.27      9840
weighted avg       0.48      0.70      0.57      9840

SVM Accuracy Score ->  69.53252032520325 for  2 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


log_loss for  3  :  0.7796954307238331 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       801
           1       0.69      1.00      0.82      6787
           2       0.00      0.00      0.00      2251

    accuracy                           0.69      9839
   macro avg       0.23      0.33      0.27      9839
weighted avg       0.48      0.69      0.56      9839

SVM Accuracy Score ->  68.98058745807501 for  3 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


log_loss for  4  :  0.7640896048820202 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       751
           1       0.70      1.00      0.82      6882
           2       0.00      0.00      0.00      2206

    accuracy                           0.70      9839
   macro avg       0.23      0.33      0.27      9839
weighted avg       0.49      0.70      0.58      9839

SVM Accuracy Score ->  69.94613273706678 for  4 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


log_loss for  5  :  0.776359916469529 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       751
           1       0.69      1.00      0.82      6778
           2       0.00      0.00      0.00      2310

    accuracy                           0.69      9839
   macro avg       0.23      0.33      0.27      9839
weighted avg       0.47      0.69      0.56      9839

SVM Accuracy Score ->  68.88911474743368 for  5 

log_loss average evaluation results of 5 classifiers:  0.7704263657254419 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Too expensive to run

# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(svc_class,x,y,cv=5, scoring = 'neg_log_loss')
# print("Cross-validated scores: ", scores)

## Test Data

In [76]:
test_df = pd.read_json("test.json")
test_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,1.0,1,79780be1514f645d7e6be99a3de696c5,2016-06-11 05:29:41,Large with awesome terrace--accessible via bed...,Suffolk Street,"[Elevator, Laundry in Building, Laundry in Uni...",40.7185,7142618,-73.9865,b1b1852c416d78d7765d746cb1b8921f,[https://photos.renthop.com/2/7142618_1c45a2c8...,2950,99 Suffolk Street
1,1.0,2,0,2016-06-24 06:36:34,Prime Soho - between Bleecker and Houston - Ne...,Thompson Street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7278,7210040,-74.0000,d0b5648017832b2427eeb9956d966a14,[https://photos.renthop.com/2/7210040_d824cc71...,2850,176 Thompson Street
2,1.0,0,0,2016-06-17 01:23:39,Spacious studio in Prime Location. Cleanbuildi...,Sullivan Street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7260,7174566,-74.0026,e6472c7237327dd3903b3d6f6a94515a,[https://photos.renthop.com/2/7174566_ba3a35c5...,2295,115 Sullivan Street
3,1.0,2,f9c826104b91d868e69bd25746448c0c,2016-06-21 05:06:02,For immediate access call Bryan.<br /><br />Bo...,Jones Street,"[Hardwood Floors, Dogs Allowed, Cats Allowed]",40.7321,7191391,-74.0028,41735645e0f8f13993c42894023f8e58,[https://photos.renthop.com/2/7191391_8c2f2d49...,2900,23 Jones Street
5,1.0,1,81062936e12ee5fa6cd2b965698e17d5,2016-06-16 07:24:27,Beautiful TRUE 1 bedroom in a luxury building ...,Exchange Place,"[Roof Deck, Doorman, Elevator, Fitness Center,...",40.7054,7171695,-74.0095,a742cf7dd3b2627d83417bc3a1b3ec96,[https://photos.renthop.com/2/7171695_089ffee2...,3254,20 Exchange Place


In [ ]:
test_df['features'] = test_df['features'].apply(lambda x: ' '.join(x))
# vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', token_pattern=r'^[a-zA-Z][a-zA-Z]+')
# w = vectorizer.fit_transform(test_df['features']);
w = vectorizer.transform(test_df['features'])
tempd = pd.DataFrame(w.toarray(), columns=vectorizer.get_feature_names())
test_df.reset_index(inplace=True, drop=True)
test_df = pd.concat([test_df, tempd], axis=1)
feat2 = ['building_id','created', 'description','display_address','features', 'listing_id','manager_id','photos', 'street_address']

In [78]:
testing = test_df[['price','bedrooms','bathrooms', 'longitude', 'latitude']]
# testing = test_df.drop(feat2, axis=1)
testing.head()

,price,bedrooms,bathrooms,longitude,latitude
0,2950,1,1.0,-73.9865,40.7185
1,2850,2,1.0,-74.0000,40.7278
2,2295,0,1.0,-74.0026,40.7260
3,2900,2,1.0,-74.0028,40.7321
4,3254,1,1.0,-74.0095,40.7054


In [ ]:
testing = scaling2.fit_transform(testing)
# pca = PCA(n_components=6)
# testing = pca.fit_transform(testing)

In [80]:
prediction = svc_class.predict_proba(testing)
prediction

array([[0.04058366, 0.70489535, 0.25452098],
       [0.07263402, 0.69724677, 0.23011921],
       [0.01959993, 0.73443482, 0.24596526],
       ...,
       [0.01955601, 0.73451161, 0.24593238],
       [0.07294412, 0.69640474, 0.23065115],
       [0.18745976, 0.63291355, 0.17962669]])

In [81]:
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.head()

,listing_id,high,medium,low
0,7142618,0.040584,0.254521,0.704895
1,7210040,0.072634,0.230119,0.697247
2,7174566,0.019600,0.245965,0.734435
3,7191391,0.072621,0.230122,0.697257
4,7171695,0.040493,0.254544,0.704963


In [ ]:
result.to_csv('result.csv', index=False) 

In [83]:
from joblib import dump, load
dump(svc_class, 'first_model.joblib')
# clf=load('first_model.joblib')

['first_model.joblib']